In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [3]:
# Install pyspark
!pip install pyspark

In [4]:
# Download the Postgres driver.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-20 01:10:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2021-12-20 01:10:57 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
# pyspark and sql interaction
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-ChallengeD2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# import data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [8]:
# verifying DFs are integers
df

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [10]:
# 1 Filter total_votes greater than or equal to 20
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [11]:
# 2 helpgul votes\total votes greater or equal to 50
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpful_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [13]:
# show new df count
print(helpful_50_df.count())

18155


In [14]:
# filter new df
vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35119071|R2BQOD1R0228FN|B00H2RSA88|     405483618|Sleep Innovations...|       Furniture|          3|           17|         26|   Y|                N|An okay product. ...|Three-stars is co...| 2015-08-27|
|         US|   44737123| RC31RUPFOHBHQ|B0125QZ50G|     350975212|Zinus Viscolatex ...|       Furniture|          4|    

In [15]:
# 4 non-vine reviews
novine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
novine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41681546| RL8D0KJ0J9L0O|B00BWC1X3S|     328960153|Zinus 14 Inch Eli...|       Furniture|          5|          152|        165|   N|                Y|A solid VICTORY f...|I've been looking...| 2015-08-31|
|         US|   16806846|R1BEINAIQFBRJC|B007I81A60|      68465765|8" Night Therapy ...|       Furniture|          5|    

In [16]:
# 5 import count function
from pyspark.sql.functions import count

In [23]:
# 5 Total number of Vine reviews 
vine_review_count = vine_review_df.count()
print("Total Number of Vine Reviews : %f" % vine_review_count)



Total Number of Vine Reviews : 136.000000


In [24]:
# 5 Total number of Non-vine reviews 
novine_review_count = novine_review_df.count()
print("Total Number of non-vine Reviews : %f" % novine_review_count)

Total Number of non-vine Reviews : 18019.000000


In [25]:
# 5 Number of 5-star Reviews for above Vine and Non-vine dataframes
star5_vine_df = vine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star Vine Reviews : %f" % star5_vine_df.count())


star5_novine_df = novine_review_df.filter(novine_review_df["star_rating"]=="5")
print("Total Number of 5-star  Non-vine Reviews : %f" % star5_novine_df.count())

Total Number of 5-star Vine Reviews : 74.000000
Total Number of 5-star  Non-vine Reviews : 8482.000000


In [26]:
# 5 Percentage of 5-star Reviews for above Vine and Non-vine dataframes
star5_vine_pct = (star5_vine_df.count()/vine_review_count)
print("Percent of 5-Star Furniture Vine Reviews : %f" % star5_vine_pct)

star5_novine_pct = (star5_novine_df.count()/novine_review_count)
print("Percent of 5-Star Furniture non-vine Reviews : %f" % star5_novine_pct)

Percent of 5-Star Furniture Vine Reviews : 0.544118
Percent of 5-Star Furniture non-vine Reviews : 0.470725


Additional analysis

In [28]:
#Percent of Vine vs non-Vine reviews
vine_all_df = df.filter(df["vine"] == "Y")
novine_all_df = df.filter(df["vine"] == "N")
vine_pct_all = (vine_all_df.count()/df.count())
novine_pct_all = (novine_all_df.count()/df.count())


print(df.count())
print(vine_all_df.count())
print(novine_all_df.count())


792113
2775
789338


In [29]:
# extract all vine reviewed products and the count
vine_prod_df = vine_all_df.select(vine_all_df["product_id"]).dropDuplicates()
vine_prod_df.show()
vine_prod_df.count()

+----------+
|product_id|
+----------+
|B00DL1AG4A|
|B00IMV6Q3Y|
|B00CPIV9BE|
|B00R465FZS|
|B007SB0554|
|B00HAR2A1I|
|B0030GJAHK|
|B00J38BRQG|
|B00FU0D2KK|
|B00WO0SQ4Q|
|B007T58QPU|
|B00H4SSFLM|
|B00CPIVB1W|
|B00CP2II9Q|
|B00T2XF8B2|
|B003PJ650C|
|B00EPRCAVG|
|B005ETZIMM|
|B00O32ZX4Y|
|B00FL0239U|
+----------+
only showing top 20 rows



421

In [31]:
# create an inner join between all the Vine Reviews df and the full df
inner_join = vine_prod_df.join(df, vine_all_df.product_id == df.product_id, how="inner").dropDuplicates()
inner_join.show()
inner_join.count()

+----------+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|product_id|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+----------+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|B00DL1AG4A|         US|   21287734|R278IHKE94IZFT|B00DL1AG4A|     987680523|Bush Furniture Ae...|       Furniture|          4|            1|          1|   Y|                N|More stylish than...|This cabinet is a...| 2015-02-02|
|B00DL1AG4A|         US|   42602916|R2MFYCTRF82JSN|B00DL1AG4A|     987680523

10689

In [32]:
# Total Vine Reviews for Vine Products
vine_new_df = inner_join.filter(inner_join["vine"] == "Y")
vine_new_df.count()

2775

In [34]:
# Total Vine Reviews for non-Vine Products
novine_new_df = inner_join.filter(inner_join["vine"] == "N")
novine_new_df.count()

7914